In [1]:
from typing import Annotated
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.prebuilt import ToolNode, tools_condition
import requests
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict

In [2]:
load_dotenv(override=True)

True

In [3]:
from langchain_community.tools import DuckDuckGoSearchResults

# Create a DuckDuckGo search tool instance
search = DuckDuckGoSearchResults()

# Make a simple query
result = search.run("What is LangGraph?")
print(result)

snippet: 2 weeks ago - LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows . It provides a set of tools and libraries that enable users to create, run and optimize large language ..., title: IBM What is LangGraph? | IBM, link: https://www.ibm.com/think/topics/langgraph, snippet: June 13, 2025 - LangGraph is an open-source framework built by LangChain that streamlines the creation and management of AI agent workflows ., title: GeeksforGeeks What is LangGraph? - GeeksforGeeks, link: https://www.geeksforgeeks.org/machine-learning/what-is-langgraph/, snippet: November 15, 2024 - LangGraph is a versatile Python library designed for stateful, cyclic, and multi-actor Large Language Model (LLM) applications . This tutorial will give you an overview of LangGraph fundamentals through hands-on examples, and the tools needed ..., title: Real Python LangGraph: Build Stateful AI Agents in Python – Real Pytho

In [4]:
from langchain_core.tools import Tool

# Convert DuckDuckGo search to a LangChain tool
search_tool = Tool(
    name="DuckDuckGo Search",
    description="Search the web using DuckDuckGo. Use this when you need to find current information about any topic. Answer precisely based on the search results.",
    func=search.run
)

print(f"Tool created: {search_tool.name}")

Tool created: DuckDuckGo Search


In [5]:
search_tool.invoke("What is the capital of Finland?")

"snippet: 5 days ago - Helsinki (Swedish: Helsingfors) is the capital and most populous city in Finland. It is on the shore of the Gulf of Finland and is the seat of southern Finland's Uusimaa region. About 690,000 people live in the municipality, with 1.3 million in the capital region and 1.6 million in the metropolitan ..., title: Wikipedia Helsinki - Wikipedia, link: https://en.wikipedia.org/wiki/Helsinki, snippet: 5 days ago - Finland, officially the Republic of Finland, is a Nordic country in Northern Europe. It borders Sweden to the northwest, Norway to the north, and Russia to the east, with the Gulf of Bothnia to the west and the Gulf of Finland to the south, opposite Estonia. Finland has a population of 5.6 million, the majority being ethnic Finns. Its capital and largest city is Helsinki ..., title: Wikipedia Finland - Wikipedia, link: https://en.wikipedia.org/wiki/Finland, snippet: September 6, 2025 - Helsinki is the capital of Finland and is its largest city. It was founded

In [6]:
tools = [search_tool]

In [7]:
# Step 1 : Define the state objects
class State(TypedDict):
    """
    State object for the LangGraph agent workflow.
    
    Attributes:
        messages (Annotated[list, add_messages]): A list of messages in the conversation.
            The add_messages annotation enables automatic message aggregation and state updates.
    """
    messages: Annotated[list, add_messages]

In [8]:
# Step 2: Start the graph builder with the state object
graph_builder = StateGraph(State)

In [9]:
llm = ChatOpenAI(model="gpt-5-nano", temperature=0.1)
llm_with_tools = llm.bind_tools(tools)